日本語での説明と予測モデルの一例をかきました。説明はアメリカンフットボールを知らない方のとりかかり向けです。  
長すぎて読まないという方は最後のセルだけみてください。TL;DR: See the last cell.  
誤りがありましたらコメントいただけますと幸いです。
  
<span style="color:crimson">2019-10-17 「ヤードラインの数値について補足」箇所に追記しました。</span>

## やること

アメリカンフットボールでは、1回のプレイごとに、攻撃側のチームが前進しようとします。  
1回のプレイで攻撃側のチームが何ヤード前進できたかを「ゲイン」とよびます（逆に後退してしまいマイナスになることもあります）。  
今回やることは、プレイのときの状況（各選手がどこにいてどちらに動いているかや、何回目の攻撃かや、天候など）から、そのプレイにおけるゲインを予測することです。  
特に今回の予測対象は、「ボールを手渡された選手が走って前進する」ようなプレイ（ランプレイ）に限られています（ボールを手渡さずに投げて前進するパスプレイなどもありますが、データに含まれているのはランプレイがとられたプレイのみであるようです）（[Overview](https://www.kaggle.com/c/nfl-big-data-bowl-2020) には、ナショナルフットボールリーグではプレイの約3分の1がランプレイであるとあります）。  
 
具体的には、予測対象のプレイに対して、-99ヤードから99ヤードまで1ヤード刻みに「ゲインがそのヤード以下である確率値」を出します（※）。  
-99ヤードも後退してしまうことはまずないので、Yards-99 はまず 0.0 になります。  
99ヤードよりも前進することもまずないので、Yards99 もまず 1.0 になります。  
この 0.0 から 1.0 までの間を適切につないだ予測分布を出すことになります（累積分布なので、距離が増加したら確率値は同じか増えます。減ることはありません）。例えば、プレイ1に対して「ゲインは絶対に0ヤードだと思う」と予測するなら、Yards-1 までは 0.0、Yards0 からは 1.0 になります。プレイ2に対して「ゲインが0ヤードか1ヤードか半々だと思う」と予測するなら、Yards0 は 0.5、Yard1 以降は 1.0 といった具合です。

|  PlayId  |  Yards-99  |  Yards-98  |  ...  |  Yards-1  |  Yards0  |  Yards1  |  ...  |  Yards98  |  Yards99  |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
|  プレイ1  |  0.0  |  0.0  |  ...  |  0.0  |  1.0  |  1.0  |  ...  |  1.0  |  1.0  |
|  プレイ2  |  0.0  |  0.0  |  ...  |  0.0  |  0.5  |  1.0  |  ...  |  1.0  |  1.0  |
　  
※ 芝生エリアの x 方向の長さは100ヤードなので（ [Data](https://www.kaggle.com/c/nfl-big-data-bowl-2020/data) のイラスト）、戦線が前進するにしても後退するにしても移動距離は100ヤード未満です。  
  
---
  
また、このコンペティションは Kaggle Kernel から専用のAPIを利用して取り組まなければなりません（ [NFL Big Data Bowl Official Starter Notebook](https://www.kaggle.com/dster/nfl-big-data-bowl-official-starter-notebook) ）。  
予測するときはイテレータを回して順々に予測します。  
ただ、このようなルールになるのはおそらく当然です。なぜなら、1回のゲーム内でプレイは何度も繰り返され続いていくので、次のプレイの開始位置（YardLine）が先にわかってしまうと現在のプレイの開始位置との差をとることでゲインはわかってしまうからです（次のプレイがランプレイでなかった場合はデータに含まれないので直接開始位置はわかりませんが、2つ以上先のプレイであっても未来の情報が手に入るのはまずいです）。  
逆にこのルールでは、直前のプレイの情報までは予測に取り込むということがありえそうです。  

## 評価指標

予測累積分布と真の累積分布の分布間の2乗誤差で評価されます（ [Evaluation](https://www.kaggle.com/c/nfl-big-data-bowl-2020/overview/evaluation) ）。  
※ $H(n - Y_m)$ は、真のゲイン $Y_m$ 未満の $n$ については $0$、 $Y_m$ 以上の $n$ については $1$ をとる、真の累積分布です。  
※ 評価データ内の各プレイに対して2乗誤差が計算され、全プレイで平均されます。  


また特筆すべきなのは、Public Leaderboard のスコアはナショナルフットボールリーグの2019年のレギュラーシーズン（2019年9月～12月）の最初の数週間のデータで算出されるのに対し、最終順位に使用されるスコアはそれ以降のデータ（ので、コンペティション開始時点でまだ行われていない試合のデータになると思います）で算出されるという点です（ [NFL Big Data Bowl Official Starter Notebook](https://www.kaggle.com/dster/nfl-big-data-bowl-official-starter-notebook) ）。  

なお、訓練データに含まれているのは2017年と2018年のシーズンのデータになります（Seasonカラム参照）。  

## 訓練データ
train.csv の各行はそのプレイでのある選手の特徴です。  
なので、あるプレイのゲインを予測するときは、その PlayId をもつレコード全てをもとにすることになるはずです。  
中には特定の選手の特徴ではなく、プレイ内で共通の特徴やゲーム内で共通の特徴もあります（確認コードは下部です）。  
プレイは時系列に並んでいます。  
以下に各カラムのメモをかきました。

```
GameId                      int64 # ゲームの識別IDです。
PlayId                      int64 # プレイの識別IDです。
Team                       object # この選手がホーム側チームかアウェイ側チームかです。
X                         float64 # ハンドオフ時刻におけるこの選手の位置の X 座標です。
Y                         float64 # ハンドオフ時刻におけるこの選手の位置の Y 座標です。
S                         float64 # ハンドオフ時刻におけるこの選手の速度です。
A                         float64 # ハンドオフ時刻におけるこの選手の加速度です。
Dis                       float64 # （おそらくスナップ時刻から）ハンドオフ時刻までのこの選手の移動距離です。
Orientation               float64 # ハンドオフ時刻におけるこの選手が向いている向きです（角度の定義はイラスト参照）。？
Dir                       float64 # ハンドオフ時刻におけるこの選手が動いている向きです（角度の定義はイラスト参照）。？
NflId                       int64 # 選手の識別IDです。
DisplayName                object # 選手の名前です。
JerseyNumber                int64 # 選手の背番号です。
Season                      int64 # 2017シーズンか2018シーズンかです。《 GameId 内共通 》
YardLine                    int64 # ヤードライン： プレイの開始位置です（X 座標ではありません）。《 PlayId 内共通 》
Quarter                     int64 # ゲーム内の第何クォーターであるかです（1クォーターは15分）。《 PlayId 内共通 》
GameClock                  object # クォーターの残り時間です。《 PlayId 内共通 》
PossessionTeam             object # どちらのチームが攻撃しているかです。《 PlayId 内共通 》
Down                        int64 # 第何ダウンの攻撃であるかです。《 PlayId 内共通 》
Distance                    int64 # 次の4ダウンの攻撃権を得るためにあと何ヤード進まなければならないかです。《 PlayId 内共通 》
FieldPosition              object # ヤードラインがどちらのチームの陣地かです。ヤードラインがちょうど50ヤードならNaNです。《 PlayId 内共通 》
HomeScoreBeforePlay         int64 # プレイ直前のホーム側チームの得点です。《 PlayId 内共通 》
VisitorScoreBeforePlay      int64 # プレイ直前のアウェイ側チームの得点です。《 PlayId 内共通 》
NflIdRusher                 int64 # ランプレイする選手の NflId です。《 PlayId 内共通 》
OffenseFormation           object # 攻撃側のフォーメーションです。《 PlayId 内共通 》
OffensePersonnel           object # 攻撃側の人員配置です。《 PlayId 内共通 》
DefendersInTheBox         float64 # ボックスといわれるエリア内にいる守備側の人数です。《 PlayId 内共通 》
DefensePersonnel           object # 守備側の人員配置です。《 PlayId 内共通 》
PlayDirection              object # このプレイが前進する向きが右方向(+x)か左方向(-x)かどちらかです。《 PlayId 内共通 》
TimeHandoff                object # ハンドオフ時刻： ランプレイする選手がボールを手渡された時刻です。《 PlayId 内共通 》
TimeSnap                   object # スナップ時刻： センターの選手がボールをスナップした時刻です。《 PlayId 内共通 》
Yards                       int64 ########## そのプレイにおけるゲイン（予測対象）です。《 PlayId 内共通 》 ##########
PlayerHeight               object # 選手の身長です（フィート - インチ）。
PlayerWeight                int64 # 選手の体重です（ポンド）。
PlayerBirthDate            object # 選手の誕生日です。
PlayerCollegeName          object # 選手の出身大学です。
Position                   object # 選手のポジションです。
HomeTeamAbbr               object # ホーム側チーム名です。《 GameId 内共通 》
VisitorTeamAbbr            object # アウェイ側チーム名です。《 GameId 内共通 》
Week                        int64 # シーズン開幕何週目かです。《 GameId 内共通 》
Stadium                    object # スタジアムです。《 GameId 内共通 》
Location                   object # スタジアムがある都市です。《 GameId 内共通 》
StadiumType                object # 屋外スタジアムかとかです。《 GameId 内共通 》
Turf                       object # 天然芝とか人工芝とかです。《 GameId 内共通 》
GameWeather                object # 天気です。《 GameId 内共通 》
Temperature               float64 # 温度です。《 GameId 内共通 》
Humidity                  float64 # 湿度です。《 GameId 内共通 》
WindSpeed                  object # 風速です。《 GameId 内共通 》
WindDirection              object # 風向です。《 GameId 内共通 》
```

### ヤードラインの数値について補足 

ヤードラインは X 座標ではなく [Data](https://www.kaggle.com/c/nfl-big-data-bowl-2020/data) のイラスト内の白い線の数値であることに注意してください。  
例えば、訓練データの1行目では、ヤードラインは「NE チーム側（ FieldPosition ）35」とあります。  
NEチームの陣地が X 座標の小さい側なら 10 + 35 = 45、X 座標の大きい側なら 110 - 35 = 75 がヤードラインの X 座標です。  
このプレイIDでの選手の X 座標の平均は約 74 なので、このクォーターでの NE チームの陣地は X 座標の大きい側であり、ヤードラインの X 座標は 75 であるとわかります。  
陣地は基本的にはクォーターごとに入れ替わります。

<span style="color:crimson">
※ 2019-10-17 追記  
ヤードライン（YardLine）の X 座標はヤードラインが攻撃側チームにとって自陣か敵陣か ＆ PlayDirection で特定できます。  
 　right (+X方向) に前進かつ自陣にいるなら左半面にいる →  10 + df['YardLine']  
 　left (-X方向) に前進かつ自陣にいるなら右半面にいる → 110 - df['YardLine']  
 　right (+X方向) に前進かつ敵陣にいるなら右半面にいる → 110 - df['YardLine']  
 　left (-X方向) に前進かつ敵陣にいるなら左半面にいる →  10 + df['YardLine']  
自陣か敵陣かは、ランプレイする選手 (NflIdRusher == NflId である選手) のチーム名を特定し  
（Team が home なら HomeTeamAbbr、away なら VisitorTeamAbbr がランプレイする選手のチーム名です）、  
それが FieldPosition と一致するかどうかで特定できます。  
例えば、訓練データで最初に NflIdRusher == NflId である行をみると、このプレイでランプレイする選手の Team は home であり、  
つまり、HomeTeamAbbr より、NEチームです。  
かつ、FieldPosition も NE チームなので、ヤードラインは攻撃側チームにとって自陣にあります。  
そして、PlayDirection は left なので、-X 方向に進んでいます。  
よって、 -X 方向に前進かつ自陣にいるので右半面にいることがわかり、110 - 35 = 75 がヤードラインの X 座標です。
</span>

## データ読み込み

まずデータを読み込みます。  
以下ではデータを読み込んだ後にプレイ内で共通の特徴やゲーム内で共通の特徴を調べています。

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

# PlayId, GameId 内で共通の値であるカラムを調べる
# 明らかに選手の特徴であるカラムは除く
feats_player = ['Team', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 
                'NflId', 'DisplayName', 'JerseyNumber', 'PlayerHeight', 'PlayerWeight', 
                'PlayerBirthDate', 'PlayerCollegeName', 'Position']

for id_ in ['PlayId', 'GameId']:
    df_ = df.drop(columns=feats_player)
    groups = df_.groupby(id_)
    s = groups.agg(lambda s: len(s.unique())).apply(max) # Id内でユニークな値の個数の最大値
    print('---------- ' + id_ + ' 内共通 ----------')
    for index, value in s.iteritems():
        if value == 1:
            print(index)

## 素朴なモデル

アメリカンフットボールのルールに基づいて素朴なモデルを考えてみます。  
アメリカンフットボールでは、攻撃側のチームは連続して4回（4ダウン）攻撃できますが、4ダウン以内に10ヤード前進しなければ攻撃権を維持できません。  
つまり、そのプレイが第何ダウンであるかでどれだけ前進するかの戦略が変わってくると思います。ダウン別のヒストグラムをかいてみます。
すると、第4ダウンにいくほどデータは少ないですが、ゲインが小さい方に偏っていることがわかります（以下）。  

In [ ]:
import numpy as np
df_play = df.drop_duplicates(subset='PlayId')
plays_down1 = df_play[df_play['Down'] == 1]
plays_down2 = df_play[df_play['Down'] == 2]
plays_down3 = df_play[df_play['Down'] == 3]
plays_down4 = df_play[df_play['Down'] == 4]
print(plays_down1.shape)
print(plays_down2.shape)
print(plays_down3.shape)
print(plays_down4.shape)

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 12, 4
rcParams['font.size'] = 16
plt.hist(plays_down1['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='r', label='down1')
plt.hist(plays_down2['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='b', label='down2')
plt.hist(plays_down3['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='g', label='down3')
plt.hist(plays_down4['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='y', label='down4')
plt.xlim(-20, 30)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.show()

第3ダウンと第4ダウンはゲインの分布がやや小さい方に偏っているようにみえます。  
第3ダウンと第4ダウンについて、攻撃権を維持するのに必要な距離が1ヤードの場合とそれより長い場合に分けてプロットしてみます。  
攻撃権を維持するのに必要な距離が1ヤードの場合が特に小さい方に偏っていることがわかります（以下）。

In [ ]:
print(plays_down3[plays_down3['Distance']==1].shape)
print(plays_down3[plays_down3['Distance']>1].shape)
print(plays_down4[plays_down4['Distance']==1].shape)
print(plays_down4[plays_down4['Distance']>1].shape)

plt.hist(plays_down3[plays_down3['Distance']==1]['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='r', label='down3_1')
plt.hist(plays_down3[plays_down3['Distance']>1]['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='b', label='down3_not1')
plt.xlim(-20, 30)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.show()

plt.hist(plays_down4[plays_down4['Distance']==1]['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='r', label='down4_1')
plt.hist(plays_down4[plays_down4['Distance']>1]['Yards'], range=(-100, 100), bins=199, density=True, cumulative=True, alpha=0.3, histtype='stepfilled', color='b', label='down4_not1')
plt.xlim(-20, 30)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.show()

上の結果をもとに、「そのプレイが第何ダウンか、3 or 4 ダウンなら残りヤードが1かどうか」のみで経験分布をあてるモデルを実装します。  
先にベンチマークとして、すべてのプレイに同じ経験分布をあてるモデルを実装します。  
訓練データのうち 2017 シーズンを訓練データ、2018 シーズンを評価データとすることにします。

In [ ]:
df_play_2017 = df_play[df_play['Season'] == 2017]
df_play_2018 = df_play[df_play['Season'] == 2018]

cd = np.histogram(df_play_2017['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_play_2017)

yard_to_index = {}
for i, y in enumerate(range(-99, 100)):
    yard_to_index[y] = i

# 理論上ありえない確率を削っておく
# Ex. 自陣35なら、ゲインは 65 以上にならないし、-35 以下にならない
# Ex. 敵陣35なら、ゲインは 35 以上にならないし、-65 以下にならない
def postprocess(cd, target_df):
    yardline = target_df['YardLine'].iloc[0]
    own = (target_df['FieldPosition'].iloc[0] == target_df['PossessionTeam'].iloc[0])
    if own:
        cd[:yard_to_index[- yardline]] = 0.0
        cd[yard_to_index[100 - yardline]:] = 1.0
    else:
        cd[:yard_to_index[- 100 + yardline]] = 0.0
        cd[yard_to_index[yardline]:] = 1.0

##### モデル0： 訓練データの経験分布をそのままあてるだけの予測 #####
def make_my_predictions_0(target_df, prediction_df):
    cd_ = np.copy(cd)
    postprocess(cd_, target_df)
    prediction_df.iloc[0,:] = cd_

df_ = df_play_2017[df_play_2017['Down'] == 1]
cd1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play_2017[df_play_2017['Down'] == 2]
cd2 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play_2017[(df_play_2017['Down'] == 3) & (df_play_2017['Distance'] == 1)]
cd3_1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play_2017[(df_play_2017['Down'] == 3) & (df_play_2017['Distance'] != 1)]
cd3_not1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play_2017[(df_play_2017['Down'] == 4) & (df_play_2017['Distance'] == 1)]
cd4_1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play_2017[(df_play_2017['Down'] == 4) & (df_play_2017['Distance'] != 1)]
cd4_not1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)

##### モデル1： 訓練データの経験分布をダウン別、残りヤード別にあてる予測 #####
def make_my_predictions_1(target_df, prediction_df):
    if target_df['Down'].iloc[0] == 1:
        cd_ = np.copy(cd1)
    elif target_df['Down'].iloc[0] == 2:
        cd_ = np.copy(cd2)
    elif (target_df['Down'].iloc[0] == 3) and (target_df['Distance'].iloc[0] == 1):
        cd_ = np.copy(cd3_1)
    elif target_df['Down'].iloc[0] == 3:
        cd_ = np.copy(cd3_not1)
    elif (target_df['Down'].iloc[0] == 4) and (target_df['Distance'].iloc[0] == 1):
        cd_ = np.copy(cd4_1)
    else:
        cd_ = np.copy(cd4_not1)
    postprocess(cd_, target_df)
    prediction_df.iloc[0,:] = cd_

2つのモデルを評価してみます。  
モデル1の方がほんの少し誤差が小さいようです。

In [ ]:
from IPython.core.display import display, HTML

# 予測結果のひな型
prediction_df = pd.DataFrame()
for y in range(-99, 100):
    prediction_df['Yards' + str(y)] = [0.0]

loss = []

for i in range(len(df_play_2018)):
    target_df_ = df_play_2018.iloc[[i],:]
    prediction_df_ = prediction_df.copy()
    make_my_predictions_0(target_df_, prediction_df_) # モデル0
    # display(HTML(prediction_df_.to_html(index=False))) # 表示してみる
    # break
    
    actual_ = np.array([0.0] * 199)
    actual_[yard_to_index[target_df_['Yards'].iloc[0]]:] = 1.0
    loss_ = (prediction_df_.iloc[0,:] - actual_) * (prediction_df_.iloc[0,:] - actual_)
    loss.append(loss_.mean())

print(np.array(loss).mean())

loss = []

for i in range(len(df_play_2018)):
    target_df_ = df_play_2018.iloc[[i],:]
    prediction_df_ = prediction_df.copy()
    make_my_predictions_1(target_df_, prediction_df_) # モデル1
    # display(HTML(prediction_df_.to_html(index=False))) # 表示してみる
    # break
    
    actual_ = np.array([0.0] * 199)
    actual_[yard_to_index[target_df_['Yards'].iloc[0]]:] = 1.0
    loss_ = (prediction_df_.iloc[0,:] - actual_) * (prediction_df_.iloc[0,:] - actual_)
    loss.append(loss_.mean())

print(np.array(loss).mean())

2017シーズンと2018シーズン全てのデータに対して経験分布を算出し直し、テストデータに実行します。

In [ ]:
### 1. データを読み込む。

# import pandas as pd
# import numpy as np
# df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
# df_play = df.drop_duplicates(subset='PlayId')

### 2. 第1ダウン、第2ダウン、第3ダウン（残り1ヤードか否か）、第4ダウン（残り1ヤードか否か）の別に分布をつくっておく。

df_ = df_play[df_play['Down'] == 1]
cd1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play[df_play['Down'] == 2]
cd2 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play[(df_play['Down'] == 3) & (df_play['Distance'] == 1)]
cd3_1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play[(df_play['Down'] == 3) & (df_play['Distance'] != 1)]
cd3_not1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play[(df_play['Down'] == 4) & (df_play['Distance'] == 1)]
cd4_1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)
df_ = df_play[(df_play['Down'] == 4) & (df_play['Distance'] != 1)]
cd4_not1 = np.histogram(df_['Yards'], bins=range(-100, 100, 1))[0].cumsum() / len(df_)

### 3. ダウンと残りヤードだけみてそのまま上の分布をあてる関数を実装する（postprocess はありえない確率の削除； 上の方のセル参照）。 

def make_my_predictions_1(target_df, prediction_df):
    if target_df['Down'].iloc[0] == 1:
        cd_ = np.copy(cd1)
    elif target_df['Down'].iloc[0] == 2:
        cd_ = np.copy(cd2)
    elif (target_df['Down'].iloc[0] == 3) and (target_df['Distance'].iloc[0] == 1):
        cd_ = np.copy(cd3_1)
    elif target_df['Down'].iloc[0] == 3:
        cd_ = np.copy(cd3_not1)
    elif (target_df['Down'].iloc[0] == 4) and (target_df['Distance'].iloc[0] == 1):
        cd_ = np.copy(cd4_1)
    else:
        cd_ = np.copy(cd4_not1)
    postprocess(cd_, target_df) # See above
    prediction_df.iloc[0,:] = cd_

### 4. テストデータに適用する。
    
from kaggle.competitions import nflrush
env = nflrush.make_env()

for (test_df, prediction_df) in env.iter_test():
    make_my_predictions_1(test_df, prediction_df)
    env.predict(prediction_df)

env.write_submission_file()